In [8]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# AnimalShelter from CRUD module 
from mongo_crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "snhu123"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    
    html.Div(
        style={'display': 'flex', 'align-items': 'center', 'justify-content': 'space-between'},
        children=[
            html.Center(html.B(html.H1('CS-340 Dashboard | Thomas Christian'))), # Unique ID
            html.Div(
                html.Img(
                    style={'height': '100px', 'width': 'auto'},
                    src='data:image/png;base64,{}'.format(encoded_image.decode())
                )
            )
        ]       
    ),
    html.Hr(),
    html.Div(
        
    dcc.Dropdown(
            id="filter-type",
            options=[
                {"label": "Water Rescue", "value": "Water"},
                {"label": "Mountain/Wilderness", "value": "Mountain/Wilderness"},
                {"label": "Disaster/Tracking", "value": "Disaster/Tracking"},
                #{"label": "Reset", "value": "Reset"} !! redundant the dropdown allows clearing
            ],
            value="Reset"
        ),
    ),
    html.Hr(),
    dash_table.DataTable(id = 'datatable-id',
                         columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data = df.to_dict('records'),
                         row_selectable="single",
                         page_action="native",
                         page_current= 0,
                         page_size= 7,
                        ),
    html.Br(),
    html.Hr(),

    html.Div(
    className='row',
    style={'display': 'flex', 'width': '100vw', 'height': '75vh'},
    children=[
        html.Div(
            id='graph-id',
            className='col',
            style={'flex': 1, 'height': '100%'}
        ),
        html.Div(
            id='map-id',
            className='col',
            style={'flex': 1, 'height': '100%'}
        )
        ]
    )
    
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')]) # Value passed from dropdown

def update_dashboard(filter_type):
    
    #Blank query or query based on specifications 
    
    query = {} 
    
    if filter_type == "Water":
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == "Mountain/Wilderness":
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == "Disaster/Tracking":
        query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    elif filter_type == "Reset" or None:
        query = {}  # No filters
    
    # Retrieve data from MongoDB
    filtered_data = pd.DataFrame.from_records(db.read(query))
    
    filtered_data.drop(columns=['_id'],inplace=True)
        
    return filtered_data.to_dict('records')
        
    
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    
     # If no data is available return an empty message
     # Dont display pie chart for all animals
    if viewData is None or len(viewData) == 0 or len(viewData) == db.collection.count_documents({}): 
        return html.Div("No data available to generate the graph.")
    
    # Create a DataFrame from data
    filtered_df = pd.DataFrame(viewData)
    
    # Ensure the DataFrame is not empty 
    if 'breed' not in filtered_df.columns or filtered_df.empty:
        return html.Div("Insufficient data or 'breed' column not found.")
    
    # Generate the pie chart dynamically based on filtered data
    pie_chart = px.pie(
        filtered_df, 
        names='breed', 
        title='Animal Breed', 
    )
    
    # Return the updated graph
    return [
        dcc.Graph(
            figure=pie_chart
        )
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        # If no columns are selected, return an empty style
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

#geolocation chart
def update_map(viewData, index):
    
    dff = pd.DataFrame.from_dict(viewData)
    
    
    if viewData is None or len(viewData) == 0:
        # Default map if no data is available
        return [
            dl.Map(
                style={'height': '100%'},
                center=[30.75, -97.48], zoom=10,
                children=[dl.TileLayer(id="base-layer-id")]
            )
        ]
    
    dff = pd.DataFrame.from_dict(viewData)

    if index is None or len(index) == 0:
        # Map without markers if no selection
        return [
            dl.Map(
                style={'height': '100%'},
                center=[30.75, -97.48], zoom=10,
                children=[dl.TileLayer(id="base-layer-id")]
            )
        ]
    
    # Use the selected row index
    row = index[0]
    
    return [
        dl.Map(
            style={'height': '100%'},
            center=[dff.iloc[row, 13], dff.iloc[row, 14]], zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                    children=[
                        dl.Tooltip(dff.iloc[row, 4]),
                        dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row, 9])])
                    ]
                )
            ]
        )
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:29058/
